# AML CLI v2 と YAML による ML Pipeline 実行

[00_LoadData.ipynb](./00_LoadData.ipynb) を実行して、あらかじめデータセットの登録をおこなってください。

> Note : データセットのアップロード・登録を CLI v2 と YAML で記述できます。

## Azure へのログイン

In [ ]:
!az login

In [3]:
!az account set -s {AZURE_SUBSCRIPTION_ID}

## AML ワークスペースの環境設定

In [1]:
my_resource_group = "{AML-RESOURCE-GROUP-NAME}"
my_workspace = "{AML-WORSPACE-NAME}"

## [Optional] AML コンピュート (クラスター) の作成

demo-cpucluster1 の名前の AML compute cluster がない場合には、下記を実行<br>
(既に存在する場合には実行不要)

In [5]:
!az ml compute create --name demo-cpucluster1 \
  --resource-group $my_resource_group \
  --workspace-name $my_workspace \
  --type amlcompute \
  --min-instances 0 \
  --max-instances 1 \
  --size Standard_D2_v2

Command group 'ml compute' is in preview and under development. Reference and support levels: https://aka.ms/CLI_refstatus
{
  "id": "/subscriptions/b3ae1c15-4fef-4362-8c3a-5d804cdeb18d/resourceGroups/TEST20220331-AML/providers/Microsoft.MachineLearningServices/workspaces/ws01/computes/demo-cpucluster1",
  "idle_time_before_scale_down": 120,
  "location": "eastus",
  "max_instances": 1,
  "min_instances": 0,
  "name": "demo-cpucluster1",
  "network_settings": {},
  "provisioning_state": "Succeeded",
  "resourceGroup": "TEST20220331-AML",
  "size": "STANDARD_D2_V2",
  "ssh_public_access_enabled": true,
  "tier": "dedicated",
  "type": "amlcompute"
}


## [Optional] Environment (環境) の作成

arima-env および deploy-env の Environment (環境) を AML 上に作成していない場合には下記を実行。<br>
(既に作成済の場合は不要)

In [3]:
%%writefile arima-env-config.yml
$schema: https://azuremlschemas.azureedge.net/latest/environment.schema.json
name: arima-env
image: mcr.microsoft.com/azureml/openmpi3.1.2-ubuntu18.04
conda_file: scripts/arima-env.yml
description: This is example

Writing arima-env-config.yml


In [4]:
!az ml environment create --file arima-env-config.yml \
  --resource-group $my_resource_group \
  --workspace-name $my_workspace

Command group 'ml environment' is in preview and under development. Reference and support levels: https://aka.ms/CLI_refstatus
{
  "conda_file": {
    "channels": [
      "anaconda",
      "conda-forge"
    ],
    "dependencies": [
      "python=3.6.9",
      {
        "pip": [
          "azureml-defaults",
          "statsmodels==0.12.2",
          "scipy",
          "scikit-learn",
          "joblib",
          "numpy",
          "pandas",
          "matplotlib",
          "seaborn",
          "numpy"
        ]
      }
    ],
    "name": "azureml_669dcef02b953becb05c12c0f43bc7d3"
  },
  "creation_context": {
    "created_at": "2022-03-31T08:11:45.465250+00:00",
    "created_by": "Tsuyoshi Matsuzaki",
    "created_by_type": "User",
    "last_modified_at": "2022-03-31T08:11:45.465250+00:00",
    "last_modified_by": "Tsuyoshi Matsuzaki",
    "last_modified_by_type": "User"
  },
  "description": "This is example",
  "id": "azureml:/subscriptions/b3ae1c15-4fef-4362-8c3a-5d804cdeb18d/resou

In [2]:
%%writefile deploy-env-config.yml
$schema: https://azuremlschemas.azureedge.net/latest/environment.schema.json
name: deploy-env
image: mcr.microsoft.com/azureml/openmpi3.1.2-ubuntu18.04
conda_file: scripts/deploy-env.yml
description: This is example

Writing deploy-env-config.yml


In [3]:
!az ml environment create --file deploy-env-config.yml \
  --resource-group $my_resource_group \
  --workspace-name $my_workspace

Command group 'ml environment' is in preview and under development. Reference and support levels: https://aka.ms/CLI_refstatus
{
  "conda_file": {
    "channels": [
      "anaconda",
      "conda-forge"
    ],
    "dependencies": [
      "python=3.6.9",
      {
        "pip": [
          "azureml-defaults"
        ]
      }
    ],
    "name": "arima-env"
  },
  "creation_context": {
    "created_at": "2022-04-15T09:06:43.672780+00:00",
    "created_by": "Tsuyoshi Matsuzaki",
    "created_by_type": "User",
    "last_modified_at": "2022-04-15T09:06:43.672780+00:00",
    "last_modified_by": "Tsuyoshi Matsuzaki",
    "last_modified_by_type": "User"
  },
  "description": "This is example",
  "id": "azureml:/subscriptions/b3ae1c15-4fef-4362-8c3a-5d804cdeb18d/resourceGroups/AML-rg/providers/Microsoft.MachineLearningServices/workspaces/ws01/environments/deploy-env/versions/1",
  "image": "mcr.microsoft.com/azureml/openmpi3.1.2-ubuntu18.04",
  "name": "deploy-env",
  "os_type": "linux",
  "reso

## パイプラインの構成

In [5]:
%%writefile pipeline_job.yml
$schema: https://azuremlschemas.azureedge.net/latest/pipelineJob.schema.json
type: pipeline
display_name: pipeline-cli
experiment_name: pipeline-cli
compute: azureml:demo-cpucluster1
jobs:
  train:
    name: train
    display_name: train
    command: >-
      python train.py
      --data_file ${{inputs.transaction_ts2013}}
      --model_folder ${{outputs.model_folder}}
    code: scripts
    inputs:
      transaction_ts2013:
        type: uri_file
        path: azureml:transaction_ts2013:1
    outputs:
      model_folder:
    environment: azureml:arima-env:1
  deploymodel:
    name: deploymodel
    display_name: deploymodel
    command: >-
      python DeployModel.py
      --model_folder ${{inputs.model_folder}}
    code: scripts
    inputs:
      model_folder: ${{parent.jobs.train.outputs.model_folder}}
    environment: azureml:deploy-env:1

Overwriting pipeline_job.yml


## パイプライン実行

In [7]:
!az ml job create --file pipeline_job.yml \
  --resource-group $my_resource_group \
  --workspace-name $my_workspace

Command group 'ml job' is in preview and under development. Reference and support levels: https://aka.ms/CLI_refstatus
'name', 'display_name', and 'experiment_name' cannot be configured for a child job within a pipeline job. These settings will be ignored.
'name', 'display_name', and 'experiment_name' cannot be configured for a child job within a pipeline job. These settings will be ignored.
Uploading scripts (0.01 MBs): 100%|█████| 6308/6308 [00:00<00:00, 142870.02it/s]


{
  "compute": "azureml:demo-cpucluster1",
  "creation_context": {
    "created_at": "2022-04-15T10:38:58.789876+00:00",
    "created_by": "Tsuyoshi Matsuzaki",
    "created_by_type": "User"
  },
  "display_name": "keen_basin_j13df547np",
  "experiment_name": "pipeline-cli",
  "id": "azureml:/subscriptions/b3ae1c15-4fef-4362-8c3a-5d804cdeb18d/resourceGroups/AML-rg/providers/Microsoft.MachineLearningServices/workspaces/ws01/jobs/keen_basin_j13df547np",
  "inputs": {},
  "jobs": {
    "deploymodel": {
      "$schema": 